# Top K

We consider a simple version of the knapsack problem, where each item is associated with a value and the task is to choose a subset of the items that maximizes the sum of the values of the items. We assume there are 10 items with the same weight 2, and the capacity of the knapsack is 15. For example,

    [2,7,3,5,2,3,8,2,1,5][1,2,3,4,5,6,9]

is a labeled example such that the first list specifies the values of the 10 items and the second list is a solution that specifies the indices of the items to be put into the knapsack. Since the capacity of the knapsack is fixed to be 15 and each item has weight 2, one can infer that the solutions always contain 7 items.
## Data Format

In dataGen.py, a class named "KsData" is defined in the following way.

KsData class has 6 attributes: train_data, test_data, valid_data, train_labels, test_labels, valid_labels.

train_data is an numpy array of size (1800, 10). It consists of 1800 data as follows 

        [
          data,
          ...,
          data
        ]
        
where data is a vector (numpy array) of length 10. For example, the data shown below  

        [2 2 1 3 1 2 8 1 5 1]
        
defines the 10 values of the 10 items.  
train_labels is an numpy array of size (1800, 10). It consists of 1800 label as follows.  

        [
          label,
          ...,
          label
        ]

where label is a vector (numpy array) of length 10, with k "1" and (10-k) "0". For example, the label shown below  

        [0 0 1 0 0 0 0 0 0 0]

means that the item 2 is chosen to be put into the knapsack.  
test_data is a numpy array of size (600, 10).   
valid_data is a numpy array of size (600, 10).  
test_labels is a numpy array of size (600, 10).  
valid_labels is a numpy array of size (600, 10).  



## Imports

In [ ]:
import sys
sys.path.append("../../")
import time

import torch

from dataGen import dataList, obsList, dataListTest, obsListTest
from network import FC
from neurasp import NeurASP

## Neural Rule + Constraint

In [ ]:
nnRule = '''
nn(in(10, k), [true, false]).
'''

constraint = '''
% define maxweight k 
#const k = 7.
% we make a mistake if the total weight of the chosen items exceeds maxweight 
:- #sum{1, I : in(I,k,true)} > k.
'''

## Neural Network Instantiation
- Instantiate neural networks.
- Define nnMapping: a dictionary that maps neural network names (i.e., strings) to the neural network objects (i.e., torch.nn.Module object)
- Define optimizers: a dictionary that specifies the optimizer for each network (we use the Adam optimizer here).

In [ ]:
m = FC(10, 50, 50, 50, 50, 50, 10)
nnMapping = {'in': m}
optimizer = {'in': torch.optim.Adam(m.parameters(), lr=0.001)}

## Create NeurASP Object

In [ ]:
NeurASPobj = NeurASP(nnRule+constraint, nnMapping, optimizer)

## Training and Testing

Note that our target is to find the set of items with maximal sum of the values, which is represented by the optimal stable models of the logic program. To find the optimal stable models instead of stable models during training, we need to specify "opt=True" in the learning function.

In [ ]:
saveModelPath = 'data/model.pt'
startTime = time.time()

for i in range(20):
    print('Epoch {}...'.format(i+1))
    time1 = time.time()
    NeurASPobj.learn(dataList=dataList, obsList=obsList, epoch=10, opt=True, smPickle='data/stableModels.pickle')
    time2 = time.time()
    NeurASPobj.testConstraint(dataListTest, obsListTest,[constraint])
    print('--- train time: %s seconds ---' % (time2 - time1))
    print('--- test time: %s seconds ---' % (time.time() - time2))
    print('--- total time from beginning: %s minutes ---' % int((time.time() - startTime)/60) )

print('Storing the trained model into {}'.format(saveModelPath))
torch.save(m.state_dict(), saveModelPath)
